In [34]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

seed = 42

In [35]:
BP_train = pd.read_pickle("dataset/BP_train_solution.pkl")
BP_test = pd.read_pickle("dataset/BP_test_solution.pkl")

BP_train = BP_train.sample(frac=1, random_state=seed)
BP_test = BP_test.sample(frac=1, random_state=seed)

X_train = BP_train.drop(columns=['score'])
Y_train = BP_train['score']

X_test = BP_test.drop(columns=['score'])
Y_test = BP_test['score']

In [ ]:
params = {
    "n_estimators": [20, 50, 100],
    "min_samples_leaf": [5, 10, 20, 30],
    "max_depth": [5, 10, 20],
}

et = ExtraTreesRegressor(random_state=seed)
gs = GridSearchCV(et, params)

gs.fit(X_train, Y_train)
print(sorted(gs.cv_results_))

y_pred = gs.predict(X_test)
df = pd.DataFrame({'y_test': Y_test, 'y_pred': y_pred})
scores = cross_val_score(et, X_train, Y_train, cv=5, scoring='r2')
print("R^2 scores:", scores)